In [ ]:
import pandas as pd
import networkx as nx
import torch

# Read CSV files
edges_df = pd.read_csv('/home/leanna/unfollow_prediction/Markov_chain/dataset/df_1.csv')
nodes_df = pd.read_csv('/home/leanna/unfollow_prediction/Markov_chain/dataset/nodes_0.csv')

# Create a NetworkX graph
G = nx.Graph()

# Add nodes with features
for index, row in nodes_df.iterrows():
    node_id = row['node_id']
    features = torch.tensor(row[['feat1','feat2','feat3','feat4','feat5','feat6','feat7','feat8','feat9','feat10','feat11','feat12','feat13','feat14','feat15','feat16','feat17','feat18','feat19']].values)
    G.add_node(node_id, feat=features)

# Add edges with label
for index, row in edges_df.iterrows():
    source = row['src_id']
    target = row['dst_id']
    label = row['label']
    G.add_edge(source, target, etype=label)

In [ ]:
import networkx as nx
from node2vec import Node2Vec
import numpy as np

# Create a NetworkX graph
# (assuming you have already created the graph G)

# Run Node2Vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4)
model = node2vec.fit(window=10, min_count=1, batch_words=4)

# Combine node embeddings to create edge embeddings
def combine_node_embeddings(u, v, emb):
    return (emb[u] + emb[v]) / 2

edge_embeddings = {}
for edge in G.edges():
    u, v = edge
    edge_embeddings[edge] = combine_node_embeddings(u, v, model.wv)

# edge_embeddings contains the edge embeddings for the NetworkX graph


In [ ]:
# Initialize the GCN and LSTM models

lstm_model = LSTMModel(8, 32, 1, 6)
# Prepare edge embeddings for LSTM
edge_embeddings = edge_embeddings.view(1, -1, 8)

# Run LSTM on edge embeddings and update the weights
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

# Dummy target, replace with actual target
target = torch.tensor([2])  # Represents class d3, with labels ranging from 0 to 5 (inclusive)

for epoch in range(100):
    output = lstm_model(edge_embeddings)
    loss = criterion(output.unsqueeze(0), target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/100], Loss: {loss.item()}")